In [1]:
import csv
import numpy as np
import random as rnd

x1_age = []
x2_sex = []
x3_cp = []
x4_trestbps = []
x5_chol = []
x6_fbs = []
x7_restecg = []
x8_thalach = []
x9_exang = []
x10_oldpeak = []
x11_slope = []
x12_ca = []
x13_thal = []
y_target = []

def read_file(file_name):

    data = csv.reader(open(file_name, newline=''))
    
    skip = True
    for row in data:
        
        if skip:
            skip = False
            continue
        
        x1_age.append(int(row[0]))
        x2_sex.append(int(row[1]))
        x3_cp.append(int(row[2]))
        x4_trestbps.append(int(row[3]))
        x5_chol.append(int(row[4]))
        x6_fbs.append(int(row[5]))
        x7_restecg.append(int(row[6]))
        x8_thalach.append(int(row[7]))
        x9_exang.append(int(row[8]))
        x10_oldpeak.append(float(row[9]))
        x11_slope.append(int(row[10]))
        x12_ca.append(int(row[11]))
        x13_thal.append(int(row[12]))
        
        y_target.append(int(row[13]))

In [2]:
read_file("../data/heart_disease_dataset_UCI.csv")

In [3]:
def create_feature_matrix():
    
    norm = np.linalg.norm(np.array(x1_age))
    Age = np.array(x1_age)/norm
    Sex = np.array(x2_sex)
    CP = np.array(x3_cp)
    norm = np.linalg.norm(np.array(x4_trestbps))
    Trestbps = np.array(x4_trestbps)/norm
    norm = np.linalg.norm(np.array(x5_chol))
    Chol = np.array(x5_chol)/norm
    FBS = np.array(x6_fbs)
    Restecg = np.array(x7_restecg)
    norm = np.linalg.norm(np.array(x8_thalach))
    Thalach = np.array(x8_thalach)/norm
    Exang = np.array(x9_exang)
    norm = np.linalg.norm(np.array(x10_oldpeak))
    Oldpeak = np.array(x10_oldpeak)/norm
    Slope = np.array(x11_slope)
    CA = np.array(x12_ca)
    Thal = np.array(x13_thal)
    
    global feature_matrix
    feature_matrix =  np.column_stack((Age,Sex,CP,Trestbps,Chol,FBS,Restecg,Thalach,Exang,Oldpeak,Slope,CA,Thal))
    
    
    
    

In [4]:
class SVM():
    
    def __init__(self, max_iter=10000, kernel_type='linear', C=1.0, epsilon=0.001, deg=2):
        self.kernels = {
            'linear' : self.kernel_linear,
            'poly' : self.kernel_poly,
            'sigmoid' : self.kernel_sigmoid,
            'rbf' : self.kernel_rbf
        }
        self.max_iter = max_iter
        self.kernel_type = kernel_type
        self.C = C
        self.epsilon = epsilon
        self.deg = deg
        
        
    def fit(self, X, y):
        n = X.shape[0]
        
        alpha = np.zeros((n))
        
        kernel = self.kernels[self.kernel_type]
        
        
        iteration = 0
        while True:
            iteration += 1
            
            alpha_prev = np.copy(alpha)
            
            for j in range(0, n):
                
                i = self.get_rnd_int(0, n-1, j) 
                
                x_i = X[i,:]
                x_j = X[j,:]
                y_i = y[i]
                y_j = y[j]
                
                k_ij = kernel(x_i, x_i) + kernel(x_j, x_j) - 2 * kernel(x_i, x_j)
                
                if k_ij == 0:
                    continue
                
                alpha_prime_j, alpha_prime_i = alpha[j], alpha[i]
                (L, H) = self.compute_L_H(self.C, alpha_prime_j, alpha_prime_i, y_j, y_i)

                self.w = self.calc_w(alpha, y, X)
                self.b = self.calc_b(X, y, self.w)

                E_i = self.E(x_i, y_i, self.w, self.b)
                E_j = self.E(x_j, y_j, self.w, self.b)

                alpha[j] = alpha_prime_j + float(y_j * (E_i - E_j))/k_ij
                alpha[j] = max(alpha[j], L)
                alpha[j] = min(alpha[j], H)

                alpha[i] = alpha_prime_i + y_i*y_j * (alpha_prime_j - alpha[j])

            diff = np.linalg.norm(alpha - alpha_prev)
            if diff < self.epsilon:
                break

            if iteration >= self.max_iter:
                print("Iteration number exceeded the max of %d iterations" % (self.max_iter))
                return
            
        self.b = self.calc_b(X, y, self.w)
        if self.kernel_type == 'linear':
            self.w = self.calc_w(alpha, y, X)
            
            
    def predict(self, X):
        return self.h(X, self.w, self.b)
    
    def calc_b(self, X, y, w):
        b_tmp = y - np.dot(w.T, X.T)
        return np.mean(b_tmp)
    
    def calc_w(self, alpha, y, X):
        return np.dot(alpha * y, X)
    
    def h(self, X, w, b):
        return np.sign(np.dot(w.T, X.T) + b).astype(int)
    
    def E(self, x_k, y_k, w, b):
        return self.h(x_k, w, b) - y_k
    
    def compute_L_H(self, C, alpha_prime_j, alpha_prime_i, y_j, y_i):
        if(y_i != y_j):
            return (max(0, alpha_prime_j - alpha_prime_i), min(C, C - alpha_prime_i + alpha_prime_j))
        else:
            return (max(0, alpha_prime_i + alpha_prime_j - C), min(C, alpha_prime_i + alpha_prime_j))
        
    def get_rnd_int(self, a,b,z):
        i = z
        cnt=0
        while i == z and cnt<1000:
            i = rnd.randint(a,b)
            cnt=cnt+1
        return i
    
    def kernel_linear(self, x1, x2):
        return np.dot(x1, x2.T)
    def kernel_poly(self, x1, x2):
        return np.dot(x1, x2.T) ** self.deg
    def kernel_sigmoid(self, x1, x2):
        return np.tanh(np.dot(x1, x2.T))
    def kernel_rbf(self, x1, x2):
        return np.exp(-np.dot(x1 - x2, x1 - x2))
    
    #Printing out the parameters of SVM
    def print_info(self):
        print("----------RUN DETAILS------------")
        print("C:",self.C)
        print("max_iter:",self.max_iter)
        print("epsilon:",self.epsilon)
        print("kernel_type:",self.kernel_type)

In [5]:
def calc_acc(y, y_hat):
    
    correct_counter = 0
    for i in range(0,len(y)):
        if(y[i] == 0 and y_hat[i] == 0):
            correct_counter = correct_counter + 1
        if(y[i] == 1 and y_hat[i] == 1):
            correct_counter = correct_counter + 1
    
    return (correct_counter/len(y))

In [6]:
def main(C, epsilon, kernel):
    
    
    
    max_iter = 10000
    kernel_type = kernel
    
    model = SVM(max_iter,kernel_type,C,epsilon)
    
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(feature_matrix, y_target, test_size = 0.25, random_state = 40)
    
    global support_vectors

    model.fit(X_train,y_train)

    y_hat = model.predict(X_test)
    
#     print(len(y_test))
#     print(len(y_hat))
    accuracy = calc_acc(y_test, y_hat)
    
    model.print_info()
    print("\nAccuracy:",accuracy)


In [7]:
create_feature_matrix()

In [8]:
# linear kernel
main(0.01,  'linear')

----------RUN DETAILS------------
C: 0.01
max_iter: 1000
kernel_type: linear

Accuracy: 0.724


In [9]:
# poly kernel
main(0.1, 'poly')

----------RUN DETAILS------------
C: 0.1
max_iter: 1000
kernel_type: poly

Accuracy: 0.652


In [10]:
# sigmoid kernel
main(1, 'sigmoid')

----------RUN DETAILS------------
C: 1
max_iter: 1000
kernel_type: sigmoid

Accuracy: 0.693


In [11]:
# rbf kernel
main(100, 0.01, 'rbf')

----------RUN DETAILS------------
C: 100
gamma: 0.01
max_iter: 1000
kernel_type: rbf

Accuracy: 0.657
